<a href="https://colab.research.google.com/github/LeoMarotta/Workshops-TTS-ASR-AlanaAI/blob/main/Workshop_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ASR Demo

This notebook will present an application of ASR in Portuguese.

## Importing packages

In [ ]:
#install
!pip install transformers
!pip install gradio
!pip install torchaudio
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.3 MB/s 
     |████████████████████████████████| 7.6 MB 36.0 MB/s 
     |████████████████████████████████| 163 kB 53.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.4 MB/s 
     |████████████████████████████████| 212 kB 37.6 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 57 kB 4.7 MB/s 
     |████████████████████████████████| 2.3 MB 38.0 MB/s 
     |████████████████████████████████| 112 kB 43.9 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 270 kB 45.1 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
    

In [ ]:
!pip freeze

absl-py==1.2.0
aeppl==0.0.33
aesara==2.7.9
aiohttp==3.8.3
aiosignal==1.2.0
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
anyio==3.6.1
appdirs==1.4.4
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
atari-py==0.2.9
atomicwrites==1.4.1
attrs==22.1.0
audioread==3.0.0
autograd==1.5
Babel==2.10.3
backcall==0.2.0
bcrypt==4.0.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.8
bokeh==2.3.3
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==2.0.8
certifi==2022.9.24
cffi==1.15.1
cftime==1.6.2
chardet==3.0.4
charset-normalizer==2.1.1
click==7.1.2
clikit==0.6.2
cloudpickle==1.5.0
cmake==3.22.6
cmdstanpy==1.0.7
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.2
cons==0.4.5
contextlib2==0.5.5
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cryptography==38.0.1
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.2.1
cycler==0.11.0
cymem==2.0.6
Cython==0.29.32
daft==0.0.4
dask==2022.2.0
da

In [ ]:
import tensorflow
import torch 
import torchaudio
from transformers import AutoModelForCTC, AutoProcessor
import gradio as gr

# Load model

In [ ]:
#load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "facebook/wav2vec2-large-xlsr-53-portuguese"
model = AutoModelForCTC.from_pretrained(model_name).to(device)
processor = AutoProcessor.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/386 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

# Funcoes

In [ ]:
#funcoes para transcrever

def load_audio(path):
  speech,sampling_rate = torchaudio.load(path)
   #stereo to mono
  speech = torch.mean(speech,0)
  #resampling (model trained on 16k audio)
  resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16_000)      
  speech = resampler.forward(speech.squeeze(0)).numpy()  
  sampling_rate = resampler.new_freq

  return speech,sampling_rate


def transcription(speech,sampling_rate,model,processor):
  features = processor(speech, sampling_rate=sampling_rate, padding=True, return_tensors="pt")
  input_values = features.input_values.to(device)
  attention_mask = features.attention_mask.to(device)
  with torch.no_grad():
    logits = model(input_values, attention_mask=attention_mask).logits
  pred_ids = torch.argmax(logits, dim=-1)
  prediction = processor.batch_decode(pred_ids)
  prediction = [pred.lower() for pred in prediction]
  prediction = prediction[0]
  return prediction

def transc(audio):
  s,sr = load_audio(audio)
  prediction = transcription(s,sr,model,processor)
  return prediction


# Transcricao

In [ ]:
import gradio as gr

gr.Interface(
    fn=transc,
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text").launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Your interface requires microphone or webcam permissions - this may cause issues in Colab. Use the External URL in case of issues.
Running on public URL: https://22177.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


/usr/local/lib/python3.7/dist-packages/gradio/processing_utils.py:216: UserWarning: Audio data is not in 16-bit integer format.Trying to convert to 16-bit int format.
  "Audio data is not in 16-bit integer format."
/usr/local/lib/python3.7/dist-packages/gradio/processing_utils.py:216: UserWarning: Audio data is not in 16-bit integer format.Trying to convert to 16-bit int format.
  "Audio data is not in 16-bit integer format."
/usr/local/lib/python3.7/dist-packages/gradio/processing_utils.py:216: UserWarning: Audio data is not in 16-bit integer format.Trying to convert to 16-bit int format.
  "Audio data is not in 16-bit integer format."
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/gradio/routes.py", line 276, in run_predict
    fn_index, raw_input, username, session_state, iterators
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 785, in process_api
    result = await self.call_function(fn_index, inputs, iterator)
  File "/usr